## Panel
pandas的panel是由items, major_xs, minor_xs构成的三维数据，方便管理三维数据。

### 字典转Panel

In [ ]:
from datetime import datetime
import pandas as pd
symbol=['600036.XSHG','600050.XSHG','601318.XSHG']
data_dict = {}
for s in symbol:
    data =  pd.read_excel('sz50.xlsx',sheetname=s, index_col='datetime')
    data_dict[s] = data.loc['2017-03-21':'2017-05-10']
PN = pd.Panel(data_dict)
print(PN)

In [ ]:
MI = PN.to_frame()
print(round(MI.head(),1))

### 修改顺序

In [ ]:
MI_tp = PN.transpose(2,1,0).to_frame()
print(MI_tp.head())

### 修改名称

In [ ]:
PN_rename = PN.rename(items={'600036.XSHG':'ZSYH','600050.XSHG':'ZGLT','601318.XSHG':'ZGPA'})

In [ ]:
print(PN_rename)

## resample抽样取周线

In [ ]:
PN_RE = PN.transpose(2,1,0).resample('W-MON',axis=1).last()
print(PN_RE)

In [ ]:
print(PN_RE.to_frame().head())

### 访问数据

### 访问items的数据

In [ ]:
print(PN_RE['close'].head())

### 访问major的数据

In [ ]:
print(PN_RE.major_xs('2017-04-10'))

### 访问minor的数据

In [ ]:
print(PN_RE.minor_xs('600036.XSHG').head())

### 访问loc使用名称索引

In [ ]:
print(PN_RE.loc[:,'2017-03-21':'2017-04-10',:].to_frame())

### 访问iloc使用位置索引

In [ ]:
print(PN_RE.iloc[2:,2:,2:])

### 处理缺失值

In [ ]:
print(PN.isnull().values.any())

In [ ]:
if PN.isnull().values.any():
    PN.fillna(method='ffill',inplace=True)

In [ ]:
print(PN.isnull().values.any())

### 多维数据计算与合并

### 计算每只股票的5日均线，并且合并成DataFrame

In [ ]:
import talib as ta

df_ma = pd.DataFrame({name: ta.abstract.MA(value, 5) for name, value in PN.iteritems()})
print(df_ma.tail())

### 计算每只股票的macd, 然后合并成MultiIndex

In [ ]:
pn_macd = pd.Panel({name: ta.abstract.MACD(value) for name, value in PN.iteritems()})
df_macd = pn_macd.transpose(2,1,0).to_frame().head()
print(df_macd)

### 用stack()将DataFrame转换multiIndex，再将两个multiIndex合并

In [ ]:
df_macd['ma'] = df_ma.stack()
print(df_macd)